# Mrinmayi Gavali


## Student ID: 012431588


### CMPE 256 - Programming Assignment 2

### Results: 


##### Number of ‘.gif’ :   73780

##### Number of ‘.jpg and .jpeg’ :   2738441

##### Number of other requests like ‘.php’ :   41966300

In [ ]:
#! pip install pyspark

In [1]:
#Import required libraries

import pyspark as spark
from pyspark import SparkContext
from operator import add
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

## Method 1: Reading the log file into  a Data Frame

In [2]:
#Create a spark session

spark = SparkSession.builder.appName("PythonWordCount").getOrCreate()

In [3]:
#Read the access_log file in a dataframe

df = spark.read.csv('access_log', sep=" ")

In [4]:
#Print the schema

df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [5]:
#View the first 10 rows of dataframe

df.head(10)

[Row(_c0='10.223.157.186', _c1='-', _c2='-', _c3='[15/Jul/2009:14:58:59', _c4='-0700]', _c5='GET / HTTP/1.1', _c6='403', _c7='202'),
 Row(_c0='10.223.157.186', _c1='-', _c2='-', _c3='[15/Jul/2009:14:58:59', _c4='-0700]', _c5='GET /favicon.ico HTTP/1.1', _c6='404', _c7='209'),
 Row(_c0='10.223.157.186', _c1='-', _c2='-', _c3='[15/Jul/2009:15:50:35', _c4='-0700]', _c5='GET / HTTP/1.1', _c6='200', _c7='9157'),
 Row(_c0='10.223.157.186', _c1='-', _c2='-', _c3='[15/Jul/2009:15:50:35', _c4='-0700]', _c5='GET /assets/js/lowpro.js HTTP/1.1', _c6='200', _c7='10469'),
 Row(_c0='10.223.157.186', _c1='-', _c2='-', _c3='[15/Jul/2009:15:50:35', _c4='-0700]', _c5='GET /assets/css/reset.css HTTP/1.1', _c6='200', _c7='1014'),
 Row(_c0='10.223.157.186', _c1='-', _c2='-', _c3='[15/Jul/2009:15:50:35', _c4='-0700]', _c5='GET /assets/css/960.css HTTP/1.1', _c6='200', _c7='6206'),
 Row(_c0='10.223.157.186', _c1='-', _c2='-', _c3='[15/Jul/2009:15:50:35', _c4='-0700]', _c5='GET /assets/css/the-associates.css H

In [32]:
#Print the total number of Rows

df.count() #This gets a different result than RDD, the total count is 44778521, use RDD to get expected result

4477843

In [33]:
#Using the following code we can determine the counts by mapping the log file to a dataframe
"""textfile2 = sc.textFile('access_log')
rdd = textfile2.map(lambda r: Row(r)).toDF(["line"])

#counts files mentioning gif
gifs = rdd.filter(col("line").like("%gif%"))

# Counts all the gifs
gifs.count()

jpgs = rdd.filter(col("line").like("%jpg%"))
jpgs.count()

jpegs = rdd.filter(col("line").like("%jpeg%"))
jpegs.count()"""

'textfile2 = sc.textFile(\'access_log\')\nrdd = textfile2.map(lambda r: Row(r)).toDF(["line"])\n\n#counts files mentioning gif\ngifs = rdd.filter(col("line").like("%gif%"))\n\n# Counts all the gifs\ngifs.count()\n\njpgs = rdd.filter(col("line").like("%jpg%"))\njpgs.count()\n\njpegs = rdd.filter(col("line").like("%jpeg%"))\njpegs.count()'

## Method 2: Map and Reduce without Filter

In [7]:
sc=spark.sparkContext

In [8]:
textfile=sc.textFile('access_log')

In [9]:
one_RDD=textfile.flatMap(lambda line: line.split(' '))

In [10]:
one_RDD.count() #Total logs

44778521

In [11]:
one_RDD=one_RDD.map(lambda file: (file,1))

In [12]:
one_RDD.count() #Total logs after map

44778521

In [13]:
one_RDD=one_RDD.reduceByKey(add)

In [28]:
output = one_RDD.collect()

c=0

#Count number of requests except jpg,jpeg and gif

for (word, count) in output:
    if('jpg' not in word and 'jpeg' not in word and 'gif' not in word):
        c += count
        
print("The number of other requests except jpg, jpeg and gif is: ",c)

The number of other requests except jpg, jpeg and gif is:  41966300


In [16]:
one_RDD.count() #After reduce function

2475646

In [17]:
one_RDD.take(15) #View the first 15 words and their frequency

[('/assets/js/the-associates.js', 2456),
 ('25960', 57),
 ('/assets/img/home-media-block-placeholder.jpg', 54),
 ('[15/Jul/2009:21:04:42', 1),
 ('[15/Jul/2009:21:24:10', 5),
 ('/displaytitle.php?id=10', 204),
 ('10011', 25),
 ('/assets/img/download-icon.gif', 219),
 ('10.82.30.199', 251),
 ('8832', 14),
 ('/assets/img/contact-us-logo.png', 33),
 ('/assets/img/google-map.gif', 1170),
 ('[16/Jul/2009:03:14:59', 5),
 ('[28/Jul/2009:06:48:35', 3),
 ('[28/Jul/2009:06:52:25', 6)]

## Method 3: Using Filter, Map and Reduce


In [19]:
spark = SparkSession.builder.appName("access_log").getOrCreate()

textfile3=sc.textFile('access_log')

#We will add a filter here 
counts=textfile3.flatMap(lambda line: line.split(' ')) \
             .filter(lambda x: ('gif' in x) or ('jpg' in x) or ('jpeg' in x)) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(add)


counts.collect() #Collect counts into an object

[('/assets/img/home-media-block-placeholder.jpg', 54),
 ('/assets/img/download-icon.gif', 219),
 ('/assets/img/google-map.gif', 1170),
 ('/assets/img/dummy/people/person-04.jpg', 10),
 ('/assets/img/dummy/campaigns/campaign-03.jpg', 16),
 ('/images/filmpics/0000/0043/weirdsville2_thumb.jpeg', 40),
 ('/images/filmpics/0000/0049/weirdsville5_thumb.jpeg', 40),
 ('/images/filmpics/0000/0047/weirdsville4_thumb.jpeg', 40),
 ('/images/filmpics/0000/0000/weirdsville1_thumb.jpeg', 1),
 ('/images/clientlogos/0000/0002/contenderlogo.gif', 22),
 ('/images/filmpics/0000/0105/Waveriders_2D_pack_jpg.jpg', 20),
 ('/images/filmpics/0000/0047/weirdsville4.jpeg', 20),
 ('/images/clientlogos/0000/0015/Arrow.jpg', 1273),
 ('/images/filmpics/0000/0131/Cyborg_She_Img_03.jpeg', 59),
 ('/images/filmpics/0000/0147/Cyborg_She_Img_02_thumb.jpeg', 3),
 ('/images/filmpics/0000/0165/MANGB8003_thumb.jpg', 329),
 ('/images/frontpagepics/0000/0008/SNDN700.jpg', 38),
 ('/images/clientlogos/0000/0009/Cine_Asia.jpg', 5048

In [24]:
output = counts.collect()

count_jpg=0
count_gif=0

for (word, count) in output:
    
    if('jpg' in word or 'jpeg' in word):
        count_jpg += count
    elif('gif' in word):
        count_gif += count  

print("The number of jpg and jpeg files are: {}, The number of gif files are: {}".format(count_jpg,count_gif))

The number of jpg and jpeg files are: 2738441, The number of gif files are: 73780


### Saving results to a text file

In [30]:
with open("Output.txt", "w") as text_file:
    text_file.write("Number of ‘.gif’ - 73780 \n")
    text_file.write("Number of ‘.jpg and .jpeg’ - 2738441 \n")
    text_file.write("Number of other requests like ‘.php’ - 41966300 \n")